In [3]:
import os
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from tmlt.analytics.privacy_budget import PureDPBudget
from tmlt.analytics.query_builder import QueryBuilder
from tmlt.analytics.session import Session

In [4]:
spark = SparkSession.builder.getOrCreate()

In [11]:
# df = spark.read.csv('simulated_hospital_data.csv', header=True, inferSchema=True)

In [5]:
# spark.sparkContext.addFile(
#     "https://tumult-public.s3.amazonaws.com/library-members.csv"
# )
hosp_df = spark.read.csv("simulated_hospital_data.csv", header=True, inferSchema=True
)

In [6]:
session = Session.from_dataframe(
    privacy_budget=PureDPBudget(3),
    source_id="hosp",
    dataframe=hosp_df
)

In [7]:
count_query = QueryBuilder("hosp").count()

In [8]:
total_count = session.evaluate(
    count_query,
    privacy_budget=PureDPBudget(epsilon=1)
)

/Users/anshusingh/.pyenv/versions/3.8.11/lib/python3.8/site-packages/pyspark/sql/pandas/functions.py:383: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [9]:
total_count.show()

+-----+
|count|
+-----+
|  102|
+-----+



In [10]:
disease_query = (
    QueryBuilder("hosp")
    .filter("Disease IN ('Mumps', 'Typhoid')")
    .count()
)
disease_count = session.evaluate(
    disease_query,
    privacy_budget=PureDPBudget(epsilon=1),
)
disease_count.show()

+-----+
|count|
+-----+
|   13|
+-----+



In [11]:
print(session.remaining_privacy_budget)


PureDPBudget(epsilon=1)


In [12]:
mean_age_query = QueryBuilder("hosp").average("Age", low=0, high=120)
mean_age = session.evaluate(
    mean_age_query,
    privacy_budget=PureDPBudget(epsilon=1)
)
mean_age.show()

+------------------+
|       Age_average|
+------------------+
|51.151515151515156|
+------------------+

